In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory

In [2]:
dataset_train_url = r"/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/train"
dataset_test_url = r"/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/test"

In [3]:
import pathlib
data_train_dir=pathlib.Path(dataset_train_url)
data_test_dir=pathlib.Path(dataset_test_url)
data_train_dir,data_test_dir

(PosixPath('/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/train'),
 PosixPath('/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/test'))

In [4]:
from tensorflow.keras.applications.efficientnet import preprocess_input

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    preprocessing_function=preprocess_input  # Preprocessing function for EfficientNetB0
)


In [5]:
train_get=datagen.flow_from_directory(
    data_train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='sparse',
    shuffle=True
)

Found 9544 images belonging to 40 classes.


In [6]:
IMG_SIZE = 224
BATCH_SIZE = 32


In [7]:
from tensorflow.keras.applications import EfficientNetB0

In [8]:
def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)

    # Add a Dense layer with 128 units and ReLU activation
    x = layers.Dense(128, activation='relu', name='dense_1')(x)

    # Output layer
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile the model
    model = keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = keras.optimizers.Adam()
    model.compile(
        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    
    return model

In [9]:
model=build_model(40)
for layer in model.layers[-10:]:
    layer.trainable = True
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss="sparse_categorical_crossentropy", metrics=["accuracy"])


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
model.fit(train_get, epochs=1)


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


299/299 ━━━━━━━━━━━━━━━━━━━━ 253s 750ms/step - accuracy: 0.6276 - loss: 1.4028


In [11]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-6), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [12]:
model.fit(train_get, epochs=1)

299/299 ━━━━━━━━━━━━━━━━━━━━ 184s 544ms/step - accuracy: 0.8436 - loss: 0.4846


In [13]:
for layer in model.layers[-25:]:
    layer.trainable = True
model.compile(optimizer=keras.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [14]:
model.fit(train_get, epochs=2)

Epoch 1/2
299/299 ━━━━━━━━━━━━━━━━━━━━ 190s 552ms/step - accuracy: 0.7941 - loss: 0.6514
Epoch 2/2
299/299 ━━━━━━━━━━━━━━━━━━━━ 173s 566ms/step - accuracy: 0.8546 - loss: 0.4347


In [24]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_efficientnet

# Function to load images from directory
def load_images_from_directory(directory, target_size=(224, 224)):
    image_list = []
    filenames = os.listdir(directory)
    
    for filename in filenames:
        img_path = os.path.join(directory, filename)
        
        # Check if it's an image file (jpg, jpeg, png)
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img = load_img(img_path, target_size=target_size)  # Load image with the target size
            img_array = img_to_array(img)  # Convert image to array
            image_list.append(img_array)
    
    # Convert list to a numpy array
    return np.array(image_list), filenames

# Load all images from the directory (replace 'data_test_dir' with your actual path)
images, filenames = load_images_from_directory(data_test_dir)

# Preprocess for ResNet50
images_resnet = preprocess_resnet(images)  # Apply ResNet50-specific preprocessing

# Preprocess for EfficientNetB0
images_efficientnet = preprocess_efficientnet(images)  # Apply EfficientNetB0-specific preprocessing

# Now, images_resnet and images_efficientnet are preprocessed images ready for ResNet50 and EfficientNetB0


In [17]:
print(f"Number of images loaded: {images.shape[0]}")
print(filenames [:10])

Number of images loaded: 3000
['02371.jpg', '00767.jpg', '02360.jpg', '00266.jpg', '02450.jpg', '01496.jpg', '01600.jpg', '00847.jpg', '00822.jpg', '00614.jpg']


In [25]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import decode_predictions


class_labels = {v: k for k, v in train_get.class_indices.items()}  # Reverse the class_indices dictionary


# Define the 40 classes from train_get + 10 specific unseen classes
desired_classes = ['horse', 'collie', 'moose', 'fox', 'sheep', 'chimpanzee', 'gorilla', 'squirrel', 'rhinoceros', 'rabbit']
all_classes = list(train_get.class_indices.keys()) + desired_classes  

# Load ResNet50 with ImageNet weights (include the top classification layer)
resnet_model = ResNet50(include_top=True, weights="imagenet")

# Make predictions using the trained model (for those seen 40 classes)
model_predictions = model.predict(images_efficientnet)

# Get the predicted class 
predicted_classes_model = tf.argmax(model_predictions, axis=-1).numpy()

# Make predictions using ResNet50 for comparison
resnet_predictions = resnet_model.predict(images_resnet)

# Decode predictions from ResNet50 to human-readable class labels
decoded_predictions = decode_predictions(resnet_predictions, top=10)  # Get top 10 predictions for each image

# Prepare a list to store the results
test_predictions = []

# Iterate over the images and compare predictions
for i in range(len(filenames)):
    img_name = filenames[i]  # Corresponding image filename

    # Get the class label predicted by trained model
    predicted_class_index_model = predicted_classes_model[i]
    predicted_class_label_model = class_labels[predicted_class_index_model]
    predicted_class_probability_model = model_predictions[i][predicted_class_index_model]  # Get probability of your model's prediction

    # Get the decoded predictions from ResNet50
    resnet_class_labels = [prediction[1] for prediction in decoded_predictions[i]]  # Extract all class labels
    resnet_class_probabilities = [prediction[2] for prediction in decoded_predictions[i]]  # Extract all probabilities

    # Initialize final prediction variable
    final_prediction = predicted_class_label_model  # Default to trained model's prediction

    # Flag to indicate whether a valid prediction from ResNet50 was found
    resnet_predicted_valid_class = False

    # Compare ResNet50's predictions with trained model's predictions
    for j in range(len(resnet_class_labels)):
        if resnet_class_labels[j] in all_classes:
            # If ResNet50 predicted a class that is either in the training set or the desired unseen classes
            resnet_predicted_valid_class = True  # We found a valid class from ResNet50

            # If ResNet50's predicted class probability is higher, take ResNet50's prediction
            if resnet_class_probabilities[j] > predicted_class_probability_model:
                final_prediction = resnet_class_labels[j]
            break  # Exit loop after selecting the prediction

    # If ResNet50 didn't predict a valid class, fallback to model's prediction
    if not resnet_predicted_valid_class:
        final_prediction = predicted_class_label_model  # Ensure the model's prediction is used if ResNet50 is not valid

    # Append the result (filename, final predicted class label)
    test_predictions.append((img_name, final_prediction))

# Save predictions to a CSV file
submission = pd.DataFrame(test_predictions, columns=['image_id', 'predicted_class_label'])
submission.to_csv("predictions.csv", index=False)

# Print a sample of the saved predictions
print(submission.head(5))


94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step
94/94 ━━━━━━━━━━━━━━━━━━━━ 13s 110ms/step
    image_id predicted_class_label
0  02371.jpg           siamese+cat
1  00767.jpg                beaver
2  02360.jpg             chihuahua
3  00266.jpg                  wolf
4  02450.jpg                   pig


In [26]:
# Save predictions to a CSV file in the correct directory for Kaggle
submission.to_csv("/kaggle/working/predictions.csv", index=False)

# Print the saved CSV file location
print("CSV file saved to: /kaggle/working/predictions.csv")


CSV file saved to: /kaggle/working/predictions.csv
